In [4]:
import random
import string
import pandas as pd
from datetime import datetime, timedelta
from faker import Faker
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

fake = Faker()

# -------------------------------
# Scrape Products Data with Selenium
# -------------------------------
opts = webdriver.ChromeOptions()
# opts.add_argument("--headless")
driver = webdriver.Chrome(options=opts)
wait = WebDriverWait(driver, 10)
products = []

driver.get("https://www.dienmayxanh.com/")
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.readmore-btn.fl-btn[href='https://www.dienmayxanh.com/flashsale']"))).click()
wait.until(EC.element_to_be_clickable((By.ID, "viewmore8673"))).click()

items = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.item")))
for item in items:
    try:
        link = item.find_element(By.TAG_NAME, "a").get_attribute("href")
        img = item.find_element(By.TAG_NAME, "img").get_attribute("src")
        name = item.find_element(By.TAG_NAME, "h3").text
        price = item.find_element(By.CSS_SELECTOR, "strong.price").text
        price = int(price.replace('.', '').replace('₫', ''))
        rating_div = driver.find_element(By.CSS_SELECTOR, "div.rating_Compare.has_quantity")
        stars = random.choice([0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])
        sold = int(float(rating_div.find_element(By.TAG_NAME, "span").text.strip()[9:13].replace(',', '.'))*1000)
        soluong = random.randint(0, 50)
        products.append({"link": link, "img": img, "name": name, "price": price, "star": stars, "sold": sold, "soluong": soluong})
    except Exception:
        continue

driver.quit()

# Assign product IDs and export to CSV
for idx, prod in enumerate(products, 1):
    prod["MaSP"] = idx
df_products = pd.DataFrame(products)
df_products = df_products[["MaSP"] + [col for col in df_products.columns if col != "MaSP"]]
df_products.to_csv("assets/data/products.csv", index=False)
print("Saved products.csv")

# -------------------------------
# Generate 1000 Random Customers
# -------------------------------
def generate_username(length=8):
    characters = string.ascii_letters + string.digits
    username = ''.join(random.choice(characters) for i in range(length))
    return username

customers = []
for i in range(1, 10001):
    customers.append({
        "MaKH": i,
        "TenKH": fake.name(),
        "DiaChi": fake.address().replace("\n", ", "),
        "SoDienThoai": fake.phone_number(),
        "Username": generate_username()
    })
df_customers = pd.DataFrame(customers)
df_customers.to_csv("assets/data/customers.csv", index=False)
print("Saved customers.csv")

# -------------------------------
# Generate 10000 Random Orders
# -------------------------------
orders = []
product_ids = df_products["MaSP"].tolist()
product_prices = dict(zip(df_products["MaSP"], df_products["price"]))
for i in range(1, 50001):
    MaDH = i
    MaKH = random.randint(1, 1000)
    MaSP = random.choice(product_ids)
    SoLuong = random.randint(1, 5)
    random_days = random.randint(0, 400)
    NgayDat = (datetime.now() - timedelta(days=random_days)).strftime("%Y-%m-%d")
    TongTien = product_prices[MaSP] * SoLuong
    orders.append({
        "MaDH": MaDH,
        "MaKH": MaKH,
        "MaSP": MaSP,
        "SoLuong": SoLuong,
        "NgayDat": NgayDat,
        "TongTien": TongTien
    })
df_orders = pd.DataFrame(orders)
df_orders.to_csv("assets/data/orders.csv", index=False)
print("Saved orders.csv")


Saved products.csv
Saved customers.csv
Saved orders.csv


In [9]:
import os

def generate_tree(startpath):
    tree = []
    
    # Traverse the directory structure
    for root, dirs, files in os.walk(startpath):
        if root == os.path.join(startpath, "node_modules"):
            continue  # Skip "node_modules" directory:
        # Calculate the depth of the current directory
        depth = root.replace(startpath, '').count(os.sep)
        
        # Add the current directory to the tree
        root_dir = root.replace(startpath, '').strip(os.sep)
        if root_dir:  # If it's not the base directory itself
            tree.append(f"{' ' * 4 * depth}└───{root_dir}")
        
        # Add subdirectories
        for dir in dirs:
            if dir == "node_modules":
                continue  # Skip "node_modules" directory
            tree.append(f"{' ' * 4 * (depth + 1)}├───{dir}")

        # Add files
        for file in files:
            tree.append(f"{''* 4 * (depth + 1)}├───{file}")

        # Add a line break after each directory
        tree.append("")

    # Remove the last empty line
    if tree:
        tree.pop()

    # Add a line break after the last directory
    if tree:
        tree.append("")

    # Add a line break after the last file
    if tree:
        tree[-1] += "\n"
            
    return tree

def save_tree_to_file(base_path, output_file="tree.txt"):
    tree_structure = generate_tree(base_path)
    
    with open(output_file, "w") as f:
        for line in tree_structure:
            f.write(line + "\n")

# Example usage
base_path = "F:/Electronic_Storage_Management"  # Change this to the base directory you want
save_tree_to_file(base_path)
